In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd
#load dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
# save file
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\adhik\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


In [6]:
##data preprocessing 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer  #convert text into vectors
from tensorflow.keras.preprocessing.sequence import pad_sequences  ##unification of text length
from sklearn.model_selection import train_test_split

#load the dataset and apply tokenizer and pad_sequence
with open('hamlet.txt','r') as file:
    text = file.read().lower()

#tokenize the text-creating index for  words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])  ##fitting have to be done in the form of list
total_words = len(tokenizer.word_index)+1
 


In [7]:
total_words

4818

In [27]:
#tokenizer.word_index

In [13]:
# create input sequences
input_sequences=[]
for line in  text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)




In [26]:
#input_sequences

In [18]:
#pad sequence , make all the sentences of equal length
max_sequence_len = max([len(x) for x in input_sequences]) ##it will tell whats the max word length
max_sequence_len

14

In [19]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [20]:
#create predictors and labels
import tensorflow as tf
#x=independent features,y=dependent features
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [21]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [23]:
y
#based on classes , some words may be repeated , some might not be repeated so  try to make a caterogy

array([ 687,    4,   45, ..., 1047,    4,  193])

In [24]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [35]:
#train lstm rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
#define model
model = Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

#compile the model
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
#train model
history = model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)


Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.0322 - loss: 7.1255 - val_accuracy: 0.0336 - val_loss: 6.7984
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.0363 - loss: 6.4319 - val_accuracy: 0.0408 - val_loss: 6.8874
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 21s 31ms/step - accuracy: 0.0493 - loss: 6.2879 - val_accuracy: 0.0493 - val_loss: 6.9416
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.0515 - loss: 6.1432 - val_accuracy: 0.0511 - val_loss: 6.9415
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.0564 - loss: 6.0083 - val_accuracy: 0.0501 - val_loss: 7.0300
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.0593 - loss: 5.8598 - val_accuracy: 0.0546 - val_loss: 7.0578
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.0676 - loss: 5.7268 - val_accuracy: 0.0635 - val_loss: 7.1406
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.0768 - loss: 5.5638 - 

In [39]:
##function to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):] #ensure sequencce length matches max_sequene lenth
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None        

In [49]:
input_text = "onni chabn"
print(f"input text:{input_text}")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"next word prediction  {next_word}")

input text:onni chabn
next word prediction  you


In [51]:
#save the model
model.save("next_word_lstm.h5")
#save tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'predicted_word_index' is not defined